# README

There is a lot of junk code in here, will clean up in the next commit.
You don't need to run most, if any, code from here.
The accompanying files are the important part of the commit.
The commit includes the following files:
<ul>
    <li>[signi-quote-count-2015_1636244638891.json,
    signi-quote-count-2016_1636246832187.json,
    signi-quote-count-2017_1636249273913.json,
    signi-quote-count-2018_1636250518608.json,
    signi-quote-count-2019_1636251729971.json,
    signi-quote-count-2020_1636237785105.json] - these files contain the number of significant quotes every speaker has for each year. A quote is classified as significant if quotemaster predicts the speaker with a certainty higher than 80%.</li>
    <li>signi-quote-count-combined_1636253317430.json - the combined counts of significant quotes over all of the years from 2015-2020</li>
    <li>american_politicians_fixed.json - contains the wiki data for all US politicians that are or have been members of the Democratic or Republican party. Only people that were alive for at least the start of 2015 have been included.</li>
    <li>top100_politicians_by_party.json - Contains the wiki data for the 100 most quoted politicians (from the previous file) from each of the parties</li>
    <li>[politician-quotes-2015_1636331534906.json,
    politician-quotes-2016_1636332058163.json,
    politician-quotes-2017_1636333168732.json,
    politician-quotes-2018_1636334221167.json,
    politician-quotes-2019_1636335010497.json,
    politician-quotes-2020_1636330658142.json] - these files contain the significant quotes of US politicians for each year.</li>
    <li>politician-quotes-combined_1636336204264.json - the significant quotes of US politicians combined over all of the years from 2015-2020. Combined with wiki data of the speakers.</li>
    <li>politician-quotes-concatenated_1636337344442.json - the significant quotes of US politicians from all of the years from 2015-2020. All quotes are combined into a single string, only the 1000 longest significant quotes were used for each politician. Combined with wiki data of the speakers.</li>
</ul>

In [1]:
import bz2
import json

In [2]:
PATH_TO_INPUT = "quotes-2020.json.bz2"
PATH_TO_OUT = "test-out-2020.json"

PATTERN_INPUT = "quotes-{}.json.bz2"

In [3]:
signi_count = 0

In [4]:
signi_quote_dict = {}

In [5]:
poli_quotes = {}
poli_people = set()

In [6]:
def iterate_through_quotes(path_to_input, path_to_out, fun_pre, fun_pre_it, fun_it, fun_post_it, fun_post):
    with bz2.open(path_to_input, 'rb') as in_file:
#         with bz2.open(PATH_TO_OUT, 'wb') as out_file:
        with open(path_to_out, 'wb') as out_file:
            fun_pre(out_file)

            i = 0
            for row in in_file:
                row = json.loads(row)
                fun_pre_it(out_file, row)
                fun_it(out_file, row)
                if fun_post_it(out_file, row, i):
                    break
                i += 1

            fun_post(out_file)

In [7]:
def iterate_through_quotes_2(path_to_input, path_to_out, fun_pre, fun_pre_it, fun_it, fun_post_it, fun_post):
    with bz2.open(path_to_input, 'rb') as in_file:
        out_file = {}
        fun_pre(out_file)

        i = 0
        for row in in_file:
            row = json.loads(row)
            fun_pre_it(out_file, row)
            fun_it(out_file, row)
            if fun_post_it(out_file, row, i):
                break
            i += 1

        fun_post(out_file)

In [8]:
def open_json(out_file):
    out_file.write('[\n'.encode('utf-8'))

In [9]:
def initialize(out_file):
    global signi_count
    global signi_quote_dict
    signi_count = 0
    signi_quote_dict = {}

In [10]:
def poli_initialize(out_file):
    global poli_quotes
    global poli_people
    global dem_list
    global rep_list
    
    poli_quotes = {}
    poli_people = set()
    
    for v in dem_list:
        poli_people.add(v['item'])
    for v in rep_list:
        poli_people.add(v['item'])

In [11]:
def open_json_and_initialize(out_file):
    open_json(out_file)
    initialize(out_file)

In [12]:
def close_json(out_file):
    out_file.write('\n]'.encode('utf-8'))

In [13]:
def copy_row(out_file, row):
    copy = json.dumps(row)
    out_file.write(copy.encode('utf-8'))

In [14]:
def count_significant_quotes(out_file, row):
    global signi_count
    global signi_quote_dict
    
    probas = row['probas']
    qids = row['qids']
    
    if (len(probas) == 0 or len(qids) == 0):
        return
    
    if (probas[0][0] == 'None'):
        return
    
    p = float(probas[0][1])
    if (p < 0.8):
        return
    
    qid = qids[0]
    
    signi_count = signi_count + 1
    signi_quote_dict[qid] = signi_quote_dict.get(qid, 0) + 1

In [15]:
def save_politician_quotes(out_file, row):
    global poli_quotes
    global poli_people
    
    probas = row['probas']
    qids = row['qids']
    
    if (len(probas) == 0 or len(qids) == 0):
        return
    
    if (probas[0][0] == 'None'):
        return
    
    p = float(probas[0][1])
    if (p < 0.8):
        return
    
    qid = qids[0]
    
    if qid not in poli_people:
        return
    
    data = {}
    data['quotation'] = row['quotation']
    data['proba'] = row['probas'][0][1]
    
    arr = poli_quotes.get(qid, [])
    arr.append(data)
    poli_quotes[qid] = arr

In [16]:
def close_row(out_file, row, i):
    if (i == 99):
        return True
    out_file.write(',\n'.encode('utf-8'))
    return False

In [17]:
def empty_fun(out_file, row):
    return

In [18]:
def empty_post(out_file):
    return

In [19]:
def empty_post_it(out_file, row, i):
    if (i == 999):
        return True
    return False

In [20]:
def write_dict_to_file(name, obj):
    millis = round(time.time() * 1000)
    name = f'{name}_{millis}.json'
    with open(name, 'wb') as f:
        output = json.dumps(obj)
        f.write(output.encode('utf-8'))

# Hello

In [21]:
iterate_through_quotes_2(PATH_TO_INPUT, PATH_TO_OUT, initialize, empty_fun, count_significant_quotes, empty_post_it, empty_post)

In [22]:
iterate_through_quotes(PATH_TO_INPUT, PATH_TO_OUT, open_json, empty_fun, copy_row, close_row, close_json)

# Test

In [23]:
# num_lines = sum(1 for line in bz2.open(PATH_TO_INPUT))

In [21]:
import sys
import os
import bz2
import time

In [22]:
def proc(input, evaluate_quote, max_length=20):
    global index
    global invalid_json_count
    global invalid_chunk_count
    global chunk_stitching
    global stitch_length
    global scrap_next
    global quote_is_open
    global quote_part
    global dat_part
    global euro_error
    global euro_count
    
    global totin
    global totout
    global prev
    global dec
    global start
    """Decompress and process a piece of a compressed stream"""
    dat = dec.decompress(input)
    got = len(dat)
    if got != 0:    # 0 is common -- waiting for a bzip2 block
        # process dat here
        
#         if (index > 10):
#             return got

        try:
            if (euro_error):
                s = (dat_part + dat).decode('utf-8')
                euro_error = False
            else:
                s = dat.decode('utf-8')
            lines = s.split('\n')

            for line in lines:
                try:
                    if (scrap_next):
                        ob = json.loads(line)
                        scrap_next = False
                        quote_is_open = False
                        chunk_stitching -= stitch_length
                    else:
                        if (quote_is_open):
                            ob = json.loads(quote_part + line)
                            quote_is_open = False
                        else:
                            ob = json.loads(line)

                    # count_significant_quotes({}, ob)
                    # parametrization
                    evaluate_quote({}, ob)
                # not a valid json string
                except ValueError:
                    if (scrap_next):
                        pass
                    else:
                        if (quote_is_open):
                            chunk_stitching += 1
                            quote_part = quote_part + line
                            stitch_length += 1

                            if (stitch_length > max_length):
                                invalid_json_count += 1
                                scrap_next = True
                        else:
                            quote_is_open = True
                            quote_part = line
                            stitch_length = 0
        except UnicodeDecodeError as e:
            if (euro_error):
                dat_part = dat_part + dat
            else:
                euro_error = True
                dat_part = dat
            
            euro_count += 1
        
        index += 1
    return got

In [23]:
def run_through_quotes(init, evaluate_quote, year, target_dict, path_to_input, name='test', chunk_size=16384):
    global index
    global invalid_json_count
    global invalid_chunk_count
    global chunk_stitching
    global stitch_length
    global scrap_next
    global quote_is_open
    global quote_part
    global dat_part
    global euro_error
    global euro_count
    
    global totin
    global totout
    global prev
    global dec
    global start
    
    size = os.path.getsize(path_to_input)
    invalid_json_count = 0
    invalid_chunk_count = 0
    chunk_stitching = 0
    stitch_length = 0
    scrap_next = False
    quote_is_open = False
    quote_part = ''
    dat_part = 0
    euro_error = False
    euro_count = 0
    
    totin = 0
    totout = 0
    prev = -1
    dec = bz2.BZ2Decompressor()
    start = time.time()
    
    init({})
    
    index = 0
    with open(path_to_input, 'rb') as f:
        for chunk in iter(lambda: f.read(chunk_size), b''):
            # feed chunk to decompressor
            got = proc(chunk, evaluate_quote)

            # handle case of concatenated bz2 streams
            if dec.eof:
                rem = dec.unused_data
                dec = bz2.BZ2Decompressor()
                got += proc(rem, evaluate_quote)

            # show progress
            totin += len(chunk)
            totout += got
            if got != 0:    # only if a bzip2 block emitted
                frac = round(1000 * totin / size)
                if frac != prev:
                    left = (size / totin - 1) * (time.time() - start)
                    print(f'\r{frac / 10:.1f}% (~{left:.1f}s left)\tyear: {year}\tnumber of speakers: {len(poli_quotes)}\tstitching: {chunk_stitching}\teuro count: {euro_count}\tinvalid json count: {invalid_json_count}\tinvalid chunk count: {invalid_chunk_count}', end='')
                    prev = frac

    # Show the resulting size.
    print(end='\r')
    print(totout, 'uncompressed bytes')

    write_dict_to_file(f'{name}-{year}', poli_quotes)

In [35]:
# year = 2020
# path_to_input = PATTERN_INPUT.format(year)

years = [2015, 2016, 2017, 2018, 2019]
# years = [2020]
for year in years:
    path_to_input = PATTERN_INPUT.format(year)
    
    run_through_quotes(
        poli_initialize, save_politician_quotes, year, poli_quotes, path_to_input, name='politician-quotes', chunk_size=1_048_576)
    print('')
    print(f'Finished compiling quotes for the year {year}')

14797890201 uncompressed bytes	number of speakers: 199	stitching: 0	euro count: 1	invalid json count: 0	invalid chunk count: 00

Finished compiling quotes for the year 2015
10584617634 uncompressed bytes	number of speakers: 200	stitching: 0	euro count: 0	invalid json count: 0	invalid chunk count: 00

Finished compiling quotes for the year 2016
24502223120 uncompressed bytes	number of speakers: 200	stitching: 33	euro count: 0	invalid json count: 0	invalid chunk count: 00

Finished compiling quotes for the year 2017
22665822486 uncompressed bytes	number of speakers: 200	stitching: 2	euro count: 1	invalid json count: 0	invalid chunk count: 00

Finished compiling quotes for the year 2018
16354868483 uncompressed bytes	number of speakers: 200	stitching: 0	euro count: 0	invalid json count: 0	invalid chunk count: 00

Finished compiling quotes for the year 2019


In [ ]:
write_dict_to_file(f'politician-quotes-{year}', poli_quotes)

In [ ]:
print(poli_quotes['Q76'][0:5])

In [ ]:
signi_quotes_file_names = [
    "signi-quote-count-2015_1636244638891.json",
    "signi-quote-count-2016_1636246832187.json",
    "signi-quote-count-2017_1636249273913.json",
    "signi-quote-count-2018_1636250518608.json",
    "signi-quote-count-2019_1636251729971.json",
    "signi-quote-count-2020_1636237785105.json"
]

In [ ]:
combined_signi_dict = {}

for file_name in signi_quotes_file_names:
    with open(file_name, 'r') as f:
        one_dict = json.load(f)
        for k in one_dict.keys():
            combined_signi_dict[k] = combined_signi_dict.get(k, 0) + one_dict[k]

In [ ]:
sorted_combined_signi_dict = {k: v for k, v in sorted(combined_signi_dict.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
write_dict_to_file('signi-quote-count-combined', sorted_combined_signi_dict)

In [ ]:
write_dict_to_file(f'signi-quote-count-{year}')

In [ ]:
print(num_lines)

In [ ]:
print(signi_count)

In [ ]:
print(len(signi_quote_dict))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

with open('signi-quote-count-combined_1636253317430.json', 'r') as f:
    dic = json.load(f)
    
    arr = np.ndarray((len(dic), 1))
    index = 0
    for k, v in dic.items():
        arr[index] = v
        index += 1
    
#     print(arr.shape)
    
df = pd.DataFrame(arr)
df[0].plot(kind="hist", logy=True, bins=5000)
plt.xlim((0, 5000))
# df[0].plot(kind="hist", loglog=True, bins=np.logspace(np.log10(1), np.log10(10e4), 100))

In [ ]:
i = 0
for k, v in sorted_combined_signi_dict.items():
    print(f'{k}: {v}')
    
    i += 1
    if (i == 10):
        break

In [ ]:
print(signi_quote_dict)

In [ ]:
SELECT DISTINCT ?item ?itemLabel 
          ?genderLabel ?citizenshipLabel ?languageLabel ?religionLabel ?ethnicLabel ?degreeLabel
          ?dateOfBirth ?placeOfBirthLabel 
#           ?nativeNameLabel ?birthNameLabel ?givenNameLabel ?familyNameLabel ?pseudonymLabel 
#           ?fatherLabel ?motherLabel ?siblingLabel ?spouseLabel ?childLabel ?numOfChild 
#           ?occupationLabel ?positionLabel ?ideologyLabel ?educatedAtLabel
          ?memberOfParty ?memberOfPartyLabel 
WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
  {
    ?item p:P106 ?statement0.
    ?statement0 (ps:P106) wd:Q82955.
    {
      ?item p:P102 ?statement1.
      ?statement1 (ps:P102) wd:Q29552.
    }
    UNION
    {
      ?item p:P102 ?statement2.
      ?statement2 (ps:P102) wd:Q29468.
    }
    MINUS {
      ?item p:P570 ?statement_3.
      ?statement_3 psv:P570 ?statementValue_3.
      ?statementValue_3 wikibase:timePrecision ?precision_3.
      FILTER(?precision_3 >= 11 )
      ?statementValue_3 wikibase:timeValue ?P570_3.
      FILTER(?P570_3 < "+2015-01-01T00:00:00Z"^^xsd:dateTime)
    }
    OPTIONAL { ?item wdt:P21 ?gender. }
    OPTIONAL { ?item wdt:P27 ?citizenship. }
    OPTIONAL { ?item wdt:P103 ?language. }
    OPTIONAL { ?item wdt:P140 ?religion. }
    OPTIONAL { ?item wdt:P172 ?ethnic. }
    OPTIONAL { ?item wdt:P512 ?degree. }
    
    OPTIONAL { ?item wdt:P569 ?dateOfBirth. }
    OPTIONAL { ?item wdt:P19 ?placeOfBirth. }
    
#     OPTIONAL { ?item wdt:P1559 ?nativeName. }
#     OPTIONAL { ?item wdt:P1477 ?birthName. }
#     OPTIONAL { ?item wdt:P735 ?givenName. }
#     OPTIONAL { ?item wdt:P734 ?familyName. }
#     OPTIONAL { ?item wdt:P742 ?pseudonym. }
    
#     OPTIONAL { ?item wdt:P22 ?father. }
#     OPTIONAL { ?item wdt:P25 ?mother. }
#     OPTIONAL { ?item wdt:P3373 ?sibling. }
#     OPTIONAL { ?item wdt:P26 ?spouse. }
#     OPTIONAL { ?item wdt:P40 ?child. }
#     OPTIONAL { ?item wdt:P1971 ?numOfChild. }
    
#     OPTIONAL { ?item wdt:P106 ?occupation. }
#     OPTIONAL { ?item wdt:P39 ?position. }
#     OPTIONAL { ?item wdt:P1142 ?ideology. }
#     OPTIONAL { ?item wdt:P69 ?educatedAt. }
    
    OPTIONAL { ?item wdt:P102 ?memberOfParty. }
  }
}

In [ ]:
with open("american_politicians_fixed.json", "r") as f:
    ampoli = json.load(f)

df = pd.DataFrame(ampoli)
df.head()

In [ ]:
df.query('itemLabel == "Arnold Schwarzenegger"')
# df["item"].duplicated().sum()

In [ ]:
def combine(x):
    d = set()
    for v in x:
        if v not in d:
            d.add(v)
    if (len(d) == 1):
        return d[0]
    arr = []
    for v in d:
        arr.append(v)
    return arr

In [ ]:
df2 = df.groupby('item').agg(combine)

In [ ]:
df2.query('item == "http://www.wikidata.org/entity/Q2685"')

In [ ]:
ampoli2 = dict()

index = 0
for row in ampoli:
    item = row['item']
    if item in ampoli2:
        tmp = ampoli2[item]
        columns = ['itemLabel', 'genderLabel', 'citizenshipLabel', 'religionLabel', 'ethnicLabel', 'degreeLabel', 'dateOfBirth', 'placeOfBirthLabel', 'memberOfParty', 'memberOfPartyLabel', 'languageLabel']
        for col in columns:
            if row.get(col, None) is None:
                continue
                
            t = tmp.get(col, None)
#             print(f'{t} {row[col]}')
            
            if t is None:
                t = row[col]
            elif isinstance(t, list):
                if row[col] not in t:
                    t.append(row[col])
            elif row[col] != t:
                t = [t, row[col]]
                
            tmp[col] = t
#             print(t)
    else:
        ampoli2[item] = row
        
#     index += 1
#     if (index == 10):
#         break

In [ ]:
df3 = pd.DataFrame(ampoli2.values())

In [ ]:
# df3.head()
df3.query('item == "http://www.wikidata.org/entity/Q2685"')

In [ ]:
with open('american_politicians_final.json', 'w') as f:
    f.write(json.dumps(list(ampoli2.values())))

In [24]:
final_poli = {}
with open('american_politicians_final.json', 'r') as f:
    final_poli = json.load(f)

In [25]:
final_poli[0:5]

[{'item': 'http://www.wikidata.org/entity/Q150851',
  'itemLabel': 'Colin Powell',
  'genderLabel': 'male',
  'citizenshipLabel': 'United States of America',
  'religionLabel': 'Episcopal Church',
  'ethnicLabel': 'African Americans',
  'degreeLabel': ['Master of Business Administration', 'Bachelor of Science'],
  'dateOfBirth': '1937-04-05T00:00:00Z',
  'placeOfBirthLabel': 'New York City',
  'memberOfParty': 'http://www.wikidata.org/entity/Q29468',
  'memberOfPartyLabel': 'Republican Party'},
 {'item': 'http://www.wikidata.org/entity/Q112534',
  'itemLabel': 'Ed Zschau',
  'genderLabel': 'male',
  'citizenshipLabel': 'United States of America',
  'dateOfBirth': '1940-01-06T00:00:00Z',
  'placeOfBirthLabel': 'Omaha',
  'memberOfParty': 'http://www.wikidata.org/entity/Q29468',
  'memberOfPartyLabel': 'Republican Party'},
 {'item': 'http://www.wikidata.org/entity/Q132071',
  'itemLabel': 'David Scott',
  'genderLabel': 'male',
  'citizenshipLabel': 'United States of America',
  'religio

In [26]:
final_poli_dict = dict()
for v in final_poli:
    qid_start = v['item'].rindex('/') + 1
    key = v['item'][qid_start:]
    
    clone = dict(v)
    clone['item'] = key
#     clone.pop('item', None)
    
    final_poli_dict[key] = clone

In [27]:
index = 0
for k, v in final_poli_dict.items():
    print(f'{k}: {v}')
    index += 1
    if index == 5:
        break

Q150851: {'item': 'Q150851', 'itemLabel': 'Colin Powell', 'genderLabel': 'male', 'citizenshipLabel': 'United States of America', 'religionLabel': 'Episcopal Church', 'ethnicLabel': 'African Americans', 'degreeLabel': ['Master of Business Administration', 'Bachelor of Science'], 'dateOfBirth': '1937-04-05T00:00:00Z', 'placeOfBirthLabel': 'New York City', 'memberOfParty': 'http://www.wikidata.org/entity/Q29468', 'memberOfPartyLabel': 'Republican Party'}
Q112534: {'item': 'Q112534', 'itemLabel': 'Ed Zschau', 'genderLabel': 'male', 'citizenshipLabel': 'United States of America', 'dateOfBirth': '1940-01-06T00:00:00Z', 'placeOfBirthLabel': 'Omaha', 'memberOfParty': 'http://www.wikidata.org/entity/Q29468', 'memberOfPartyLabel': 'Republican Party'}
Q132071: {'item': 'Q132071', 'itemLabel': 'David Scott', 'genderLabel': 'male', 'citizenshipLabel': 'United States of America', 'religionLabel': 'Baptists', 'ethnicLabel': 'African Americans', 'dateOfBirth': '1945-06-27T00:00:00Z', 'placeOfBirthLabe

In [ ]:
final_df = pd.DataFrame(final_poli)

In [ ]:
final_df.head(10)
# bernie = final_df.query('item == "http://www.wikidata.org/entity/Q359442"')
# for v in bernie['memberOfPartyLabel']:
#     print(v)
    
# final_df.query('item == "http://www.wikidata.org/entity/Q359442"')

In [ ]:
only_keys = set()
for v in final_df['item']:
    only_keys.add(v[v.rindex('/') + 1:])

In [28]:
with open('signi-quote-count-combined_1636253317430.json', 'r') as f:
    combined_quote_count_dict = json.load(f)
    
#     arr = np.ndarray((len(dic), 1))
#     index = 0
#     for k, v in dic.items():
#         arr[index] = v
#         index += 1

In [29]:
dem_list = []
rep_list = []

CAP_TARGET = 100
DEM_PARTY = "http://www.wikidata.org/entity/Q29552"
REP_PARTY = "http://www.wikidata.org/entity/Q29468"

for v in combined_quote_count_dict:
#     row = final_df.query(f'item == "http://www.wikidata.org/entity/{v}"')
    row = final_poli_dict.get(v, None)
    
    # Could not find person in the politician dictionary
    if row is None:
        continue
    
    memberOfParty = row.get('memberOfParty', None)
    if memberOfParty is None:
        continue
    
    # Cast to one element list if not already a list
    if isinstance(memberOfParty, list) == False:
        memberOfParty = [memberOfParty]
    
    # Check membership
    if DEM_PARTY in memberOfParty:
        if REP_PARTY in memberOfParty:
            # member of both parties, don't know how to handle
#             print(row)
            continue

        if len(dem_list) < CAP_TARGET:
            dem_list.append(row)
    elif REP_PARTY in memberOfParty:
        if len(rep_list) < CAP_TARGET:
            rep_list.append(row)
    
    if len(dem_list) == CAP_TARGET and len(rep_list) == CAP_TARGET:
        break

In [30]:
print(dem_list[0:5])

[{'item': 'Q76', 'itemLabel': 'Barack Obama', 'genderLabel': 'male', 'citizenshipLabel': 'United States of America', 'languageLabel': 'English', 'religionLabel': ['Congregationalist polity', 'United Church of Christ', 'Protestantism', 'congregational church'], 'ethnicLabel': 'African Americans', 'degreeLabel': ['Bachelor of Arts', 'Juris Doctor'], 'dateOfBirth': '1961-08-04T00:00:00Z', 'placeOfBirthLabel': 'Kapiolani Medical Center for Women and Children', 'memberOfParty': 'http://www.wikidata.org/entity/Q29552', 'memberOfPartyLabel': 'Democratic Party'}, {'item': 'Q6294', 'itemLabel': 'Hillary Clinton', 'genderLabel': 'female', 'citizenshipLabel': 'United States of America', 'languageLabel': 'English', 'religionLabel': 'Methodism', 'degreeLabel': ['Juris Doctor', "bachelor's degree"], 'dateOfBirth': '1947-10-26T00:00:00Z', 'placeOfBirthLabel': 'Edgewater Hospital', 'memberOfParty': 'http://www.wikidata.org/entity/Q29552', 'memberOfPartyLabel': 'Democratic Party'}, {'item': 'Q359442', 

In [31]:
for index in range(CAP_TARGET):
    dem = dem_list[index]
    rep = rep_list[index]
#     demv = list(dem['itemLabel'].values())[0]
#     repv = list(rep['itemLabel'].values())[0]
    demv = dem['itemLabel']
    demq = dem['item']
    demc = str(combined_quote_count_dict.get(demq, -1))
    repv = rep['itemLabel']
    repq = rep['item']
    repc = str(combined_quote_count_dict.get(repq, -1))
    print(f"{index + 1:3} {demv:30} {demq:10} {demc:20} {repv:30} {repq:10} {repc:10}")
    index += 1
    
    if index % 10 == 0:
        print()

  1 Barack Obama                   Q76        59650                Donald Trump                   Q22686     201293    
  2 Hillary Clinton                Q6294      28126                Mike Pompeo                    Q473239    19402     
  3 Bernie Sanders                 Q359442    27652                Lindsey Graham                 Q22212     17352     
  4 Nancy Pelosi                   Q170581    20783                Mike Pence                     Q24313     16511     
  5 Andrew Cuomo                   Q11673     20165                Mitch McConnell                Q355522    16057     
  6 Chuck Schumer                  Q380900    18489                Marco Rubio                    Q324546    15876     
  7 Joe Biden                      Q6279      17946                Ted Cruz                       Q2036942   15728     
  8 Bill de Blasio                 Q4911497   16996                John McCain                    Q10390     11362     
  9 Elizabeth Warren               Q4347

In [ ]:
combined_quote_count_dict.get('Q7595813')

In [ ]:
obj = {"dem": dem_list, "rep": rep_list}
with open('top100_politicians_by_party.json', 'w') as f:
    f.write(json.dumps(obj))

In [ ]:
with open('top100_politicians_by_party.json', 'r') as f:
    data = json.load(f)

test_dem_df = pd.DataFrame(data["dem"])
test_rep_df = pd.DataFrame(data["rep"])

# test_dem_df.head()

In [34]:
with open('politician-quotes-2020_1636330658142.json', 'r') as f:
    data = json.load(f)

print(len(data['Q76']))

538


In [40]:
poli_quote_files = [
    "politician-quotes-2015_1636331534906.json",
    "politician-quotes-2016_1636332058163.json",
    "politician-quotes-2017_1636333168732.json",
    "politician-quotes-2018_1636334221167.json",
    "politician-quotes-2019_1636335010497.json",
    "politician-quotes-2020_1636330658142.json"
]

poli_quotes_combined = {}

both_parties = dem_list + rep_list
for v in both_parties:
    copy = dict(v)
    copy['quotations'] = []
    
    poli_quotes_combined[v['item']] = copy

for pqf in poli_quote_files:
    with open(pqf, 'r') as f:
        quotes = json.load(f)
        
        for k in quotes.keys():
            poli_quotes_combined[k]['quotations'] += quotes[k]

write_dict_to_file('politician-quotes-combined', poli_quotes_combined)

In [41]:
for k in poli_quotes_combined.keys():
    value = poli_quotes_combined[k]
    print(f'{k} -> {len(value["quotations"])}')

Q76 -> 59650
Q6294 -> 28126
Q359442 -> 27652
Q170581 -> 20783
Q11673 -> 20165
Q380900 -> 18489
Q6279 -> 17946
Q4911497 -> 16996
Q434706 -> 16937
Q22316 -> 12973
Q350843 -> 10640
Q7173106 -> 9692
Q1077594 -> 7642
Q2023708 -> 7601
Q10853588 -> 7482
Q1135767 -> 7236
Q152451 -> 6791
Q461391 -> 6725
Q298443 -> 6435
Q55223040 -> 6350
Q22237 -> 6221
Q260042 -> 6202
Q529344 -> 5990
Q359888 -> 5732
Q5386557 -> 5667
Q230733 -> 5637
Q1100710 -> 5550
Q22222 -> 5385
Q6221385 -> 5384
Q1124 -> 5333
Q434804 -> 5331
Q11963798 -> 5324
Q263725 -> 5166
Q381880 -> 5150
Q15126384 -> 4952
Q3518911 -> 4781
Q13133 -> 4622
Q16885910 -> 4507
Q538868 -> 4306
Q1077115 -> 4229
Q888132 -> 4075
Q7287916 -> 3978
Q1855840 -> 3874
Q7793121 -> 3838
Q22260 -> 3796
Q430518 -> 3471
Q461727 -> 3439
Q970720 -> 3371
Q923242 -> 3338
Q59315 -> 3319
Q516515 -> 3313
Q934898 -> 3293
Q16751186 -> 3226
Q32620 -> 3219
Q23685 -> 3109
Q1282411 -> 3105
Q723295 -> 3086
Q56486914 -> 3045
Q5607720 -> 3037
Q513091 -> 2966
Q16189318 -> 2915
Q

In [69]:
poli_quotes_concat = {}

with open('politician-quotes-combined_1636336204264.json', 'r') as f:
    poli_quotes_concat = json.load(f)

In [70]:
for k in poli_quotes_concat.keys():
    elem = poli_quotes_concat[k]
    
    elem['quotations'].sort(key = lambda x: len(x['quotation']), reverse = True)
    concat = ''
    for i in range(1000):
        concat += elem['quotations'][i]['quotation']
    
    elem['quotations'] = concat

In [71]:
write_dict_to_file('politician-quotes-concatenated', poli_quotes_concat)

In [ ]:
test_dem_df.head()

In [ ]:
test_rep_df.head()

In [ ]:
i = 0

with bz2.open(PATH_TO_INPUT, 'rb') as in_file:
#     with bz2.open(PATH_TO_OUT, 'wb') as out_file:
    with open(PATH_TO_OUT, 'wb') as out_file:
        out_file.write('[\n'.encode('utf-8'))
        for row in in_file:
            row = json.loads(row)
#             print(row)
            
            out_row = json.dumps(row)
#             out_row = '{"hello": "world"}'
            out_row = out_row + (',\n' if i < 9 else '\n')
            out_file.write(out_row.encode('utf-8'))
            
            i+=1
            if (i == 10):
                out_file.write(']'.encode('utf-8'))
                break